# PGA Tour Data Retrieval
This notebook is for working with PGA_Data folder for retreiving revelant PGA Tour data that has been webscraped from [pgatour.com](https://www.pgatour.com/).

In [91]:
# General libraries
import pandas as pd
import numpy as np
from importlib import reload
import difflib

from bs4 import BeautifulSoup
import requests
import re
import json
import time

# Self made libraries
import pga_data.scrape
import pga_data.mapping

# Reload libraries
reload(pga_data.mapping)
reload(pga_data.scrape)

<module 'pga_data.scrape' from 'c:\\Users\\jborman\\Documents\\Python\\dfs_golf\\pga_data\\scrape.py'>

## Read the mapping to load json data

In [2]:
m = pga_data.mapping.mapping()

## Find Specific Player Stats
Webscrape to perform this task

In [3]:
# Loop thru all players
# Find their tour options and seasons
# Find their courses played
# Personal stats (ie Height, Weight, Age, etc)

m.player_meta[:3]
m.stat_meta[2]
# m.tourney_meta.keys()


{'stat name': 'SG: Off the Tee',
 'stat id': '02567',
 'stat category': 'Shots Gained',
 'years': [2004,
  2005,
  2006,
  2007,
  2008,
  2009,
  2010,
  2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2020,
  2021],
 'tournament option': True}

In [95]:
# ['SG: Total',
#  'SG: Tee to Green',
#  'SG: Off the Tee',
#  'SG: Approach the Green',
#  'SG: Around the Green',
#  'SG: Putting']

# driving distance
st = pga_data.scrape.stat('driving distance')
st.pull()
distance = st.data.rename({'avg':'distance'}, axis=1)

# shots gained ott
st = pga_data.scrape.stat('SG: Off the Tee')
st.pull()
sg_ott = st.data.rename({'avg':'sg_ott'}, axis=1)

# Driving Accuracy Percentage
st = pga_data.scrape.stat('Driving Accuracy Percentage')
st.pull()
accuracy = st.data.rename({'pct':'fw_hit'}, axis=1)

# smash factor
st = pga_data.scrape.stat('smash factor')
st.pull()
smash_factor = st.data.rename({'avg':'smash'},axis=1)

data successfully pulled!
data successfully pulled!
data successfully pulled!
data successfully pulled!


In [103]:
import seaborn as sns
import matplotlib.pyplot as plt
import re

cols = ['player_name','distance','sg_ott','fw_hit','smash']
sg_dis = distance.merge(sg_ott, on='player_name', how='inner', suffixes=['_dis','_sg'])
sg_dis = sg_dis.merge(accuracy, on='player_name', how='left')
sg_dis = sg_dis.merge(smash_factor, on='player_name', how='left', suffixes=['_fw','_sf'])
# sg_dis = sg_dis[cols]

# _=sns.scatterplot(data=sg_dis, x='RANK_LAST_WEEK_dis', y='RANK_LAST_WEEK_sg')
# sg_dis.describe()
# sg_dis.sort_values('sg_ott',ascending=False).tail(20)
sg_dis.iloc[0,:]

rank_this_week_dis                1
rank_last_week_dis                1
player_name           Cameron Champ
rounds_dis                       42
distance                      322.6
total_distance                27102
total_drives                     84
rank_this_week_sg                 9
rank_last_week_sg                10
rounds_sg                        42
sg_ott                         0.69
total_sg_ott                 22.772
measured_rounds                  33
rank_this_week_fw              T140
rank_last_week_fw               142
rounds_fw                        42
fw_hit                        57.49
fairways_hit                    334
possible_fairways               581
rank_this_week_sf                T8
rank_last_week_sf               T12
rounds_sf                        42
smash                         1.511
total_smash_factor          114.845
total_attempts                   76
highest_value                 1.534
lowest_value                  1.455
Name: 0, dtype: object

## Make Meta Json Files

In [7]:
reload(pga_data.mapping)
reload(pga_data.scrape)

mm = pga_data.mapping.make_meta()


In [8]:
import json
with open('pga_data/data_files/stat_meta.json') as f:
    stats = json.load(f)

In [36]:
print([d['stat name'] for d in stats if d['stat category']=='Off the Tee'][:20])
print([d['stat name'] for d in stats if d['stat category']=='Off the Tee'][-20:])
# set([d['stat category'] for d in stats])

['Driving Distance', 'Longest Drives', 'Driving Percentage 320+', 'Driving Percentage 300-320', 'Driving Percentage 300+', 'Driving Percentage 280-300', 'Driving Percentage 260-280', 'Driving Percentage 240-260', 'Driving Percentage <=240', 'Percentage of Yardage Covered by Tee Shots', 'Percentage of Yardage Covered by Tee Shots Par 4s', 'Percentage of Yardage Covered by Tee Shots Par 5s', 'Driving Accuracy Percentage', 'Rough Tendency', 'Left Rough Tendency', 'Right Rough Tendency', 'Left Rough Tendency (RTP Score)', 'Right Rough Tendency (RTP Score)', 'Fairway Bunker Tendency', 'Missed Fairway Percentage']
['Hit Fairway Percentage', 'Distance From Edge of Fairway', 'Distance From Center of Fairway', 'Left Tendency', 'Right Tendency', 'Good Drive Percentage', 'Total Driving', 'Ball Striking', 'Club Head Speed', 'Ball Speed', 'Smash Factor', 'Launch Angle', 'Spin Rate', 'Distance to Apex', 'Apex Height', 'Hang Time', 'Carry Distance', 'Carry Efficiency', 'Total Distance Efficiency', 'T

## Practice With SQL
